In [1]:
!pip install sentence_transformers

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.5-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/249.1 kB ? eta -:--:--
   ------------------------------------ --- 225.3/249.1 kB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 249.1/249.1 kB 5.1 MB/s eta 0:00:00
Using cached huggingface_hub-0.24.6-py3-none-any.whl (417 kB)
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.4/199.4 MB 11.2 MB/s eta 0:00:18
   ---------------------------------------- 0.9/199.4 MB 11.9 MB/s eta 0:00:17
   ---------------------------------------- 1.4/199.4 MB 11.1 MB/s eta 0:00:18
   ---------------------------------------- 1.7/199.4 MB 10.0 MB/s eta 0:00:20
   ------------------------

In [2]:
import urllib.request
import pandas as pd

In [1]:
from sentence_transformers import SentenceTransformer

: 

In [4]:
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [7]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

c:\Users\humming\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from langchain_community.embeddings import GPT4AllEmbeddings
gpt4all_embd = GPT4AllEmbeddings()

Downloading: 100%|██████████| 45.9M/45.9M [00:05<00:00, 8.39MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 432MiB/s]


In [6]:
from huggingface_hub import hf_hub_download
model_name_or_path = "C:/Users/humming/Downloads/eeve-korean-instruct-10.8b-v1.0.Q4_K_M.gguf"
model_basename = "ggml-model-Q4_K_M.gguf" # file name

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
print(model_path)

ggml-model-Q4_K_M.gguf:   0%|          | 0.00/6.51G [00:00<?, ?B/s]

C:\Users\humming\.cache\huggingface\hub\models--heegyu--EEVE-Korean-Instruct-10.8B-v1.0-GGUF\snapshots\9bf4892cf2017362dbadf99bd9a3523387135362\ggml-model-Q4_K_M.gguf


c:\Users\humming\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\humming\.cache\huggingface\hub\models--heegyu--EEVE-Korean-Instruct-10.8B-v1.0-GGUF. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import StreamingStdOutCallbackHandler
import pandas as pd 
import time
# GPT4All 모델 파일 경로 설정
path = 'ggml-model-Q4_K_M.gguf'

# ChatPromptTemplate 구성


lcpp_llm = GPT4All(
    model=path,
    n_threads=6,

)

# # Parquet 파일 읽기
# data = pd.read_parquet("C:/Users/humming/Downloads/GPT2_Chatbot/train-00000-of-00001-21df739eb88d711e.parquet")

# # 데이터 확인
# print(data.head())


In [16]:
prompt_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nHuman: {prompt}\nAssistant:\n"
text = '한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.\n\n(A) 경성\n(B) 부산\n(C) 평양\n(D) 서울\n(E) 전주'

prompt = prompt_template.format(prompt=text)
start = time.time()
response = lcpp_llm(
    prompt=prompt,
    max_tokens=256,
    top_p=0.95,
    top_k=50,
    stop = ['</s>'], # Dynamic stopping when such token is detected.

)

print(time.time() - start)
print(response)

20.451958417892456
한국에는 두 개의 특별시가 있습니다, 하나는 서울이고 다른 하나는 세종입니다. 하지만 일반적으로 한국의 수도로 인정받는 곳은 서울이죠. 따라서 정답은 (D) 서울입니다.


In [3]:
lcpp_llm.get_token_ids('안녕')


: 

In [6]:
from langchain_community.embeddings import GPT4AllEmbeddings

gpt4all_embd = GPT4AllEmbeddings(model_name='ggml-model-Q4_K_M.gguf',gpt4all_kwargs = {'allow_download': 'True'})


In [12]:
!pip install transformers

In [13]:
gpt4all_embd.get_token_ids(['안녕?'])

AttributeError: 'GPT4AllEmbeddings' object has no attribute 'get_token_ids'

In [11]:
data = pd.read_parquet("C:/Users/humming/Downloads/GPT2_Chatbot/train-00000-of-00001-21df739eb88d711e.parquet")


                                         instruction  \
0                   양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?   
1                              스웨터의 유래는 어디에서 시작되었나요?   
2  토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  \n\n토성의 고리는 얼음과 ...   
3  화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...   
4  '사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...   

                                              output  \
0  양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...   
1  스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...   
2  토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...   
3  화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...   
4  '사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...   

                                                 url  
0  https://kin.naver.com/qna/detail.naver?d1id=11...  
1  https://kin.naver.com/qna/detail.naver?d1id=11...  
2  https://kin.naver.com/qna/detail.naver?d1id=11...  
3  https://kin.naver.com/qna/detail.naver?d1id=5&...  
4  https://kin.naver.com/qna/detail.naver?d1id=11..

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
# 코사인 유사도 구하기
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

def return_similar_answer(input):
    # 입력 텍스트를 임베딩 벡터로 변환
    embedding = model.encode(input)
    
    # `train_data` 데이터프레임의 각 행에 대해 유사도 점수를 계산
    # lambda 함수는 각 행의 'embedding' 열과 입력 임베딩 벡터 간의 코사인 유사도를 계산
    train_data['score']=train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    
    # 가장 높은 유사도 점수를 가진 행의 인덱스를 찾고, 해당 행의 'A' 열의 값을 반환
    # idxmax()는 score 열에서 최대값의 인덱스를 반환합니다
    # loc[]는 해당 인덱스의 행을 선택합니다
    return train_data.loc[train_data['score'].idxmax()]['A']


In [15]:
lcpp_llm.generate(["안녕"])

Exception ignored on calling ctypes callback function: <function LLModel._callback_decoder.<locals>._raw_callback at 0x000001AD23986480>
Traceback (most recent call last):
  File "c:\Users\humming\anaconda3\Lib\site-packages\gpt4all\_pyllmodel.py", line 573, in _raw_callback
    def _raw_callback(token_id: int, response: bytes) -> bool:

KeyboardInterrupt: 


In [7]:
# 각 행에 대해 model.encode(row.Q)를 호출하여 임베딩 벡터를 생성 -> Q열의 각 행을 인코딩해라
data['embedding'] = data.apply(lambda row: llm.em(row.instruction), axis = 1)

LookupError: unknown encoding: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

In [ ]:
train_data.to_csv('Gpt_ChatBotData.csv')
train_data

In [9]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [12]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
# 코사인 유사도 구하기
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

def return_similar_answer(input):
    # 입력 텍스트를 임베딩 벡터로 변환
    embedding = model.encode(input)
    
    # `train_data` 데이터프레임의 각 행에 대해 유사도 점수를 계산
    # lambda 함수는 각 행의 'embedding' 열과 입력 임베딩 벡터 간의 코사인 유사도를 계산
    train_data['score']=train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    
    # 가장 높은 유사도 점수를 가진 행의 인덱스를 찾고, 해당 행의 'A' 열의 값을 반환
    # idxmax()는 score 열에서 최대값의 인덱스를 반환합니다
    # loc[]는 해당 인덱스의 행을 선택합니다
    return train_data.loc[train_data['score'].idxmax()]['A']


In [ ]:
return_similar_answer('12시 땡!')

In [ ]:
return_similar_answer('너 말 잘한다')

In [ ]:
!pip install --upgrade pandas pyarrow


In [3]:
import pandas as pd 

data = pd.read_parquet("C:/Users/humming/Downloads/GPT2_Chatbot/train-00000-of-00001-21df739eb88d711e.parquet")

In [6]:
data

,instruction,output,url
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,https://kin.naver.com/qna/detail.naver?d1id=11...
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,https://kin.naver.com/qna/detail.naver?d1id=11...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",https://kin.naver.com/qna/detail.naver?d1id=11...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,https://kin.naver.com/qna/detail.naver?d1id=5&...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",https://kin.naver.com/qna/detail.naver?d1id=11...
...,...,...,...
21150,왕의 우물이란 무엇인가요?,"왕의 우물, 즉 왕실에서 쓰던 우물은 어정, 혹은 왕정이라고 부르며 한자로는 御井,...",https://kin.naver.com/qna/detail.naver?d1id=11...
21151,한 달의 첫째 주가 어떻게 정의되는가?\n매달 첫째주가 되는 기준은 무엇인지 궁금합...,한 주의 시작이 일요일인지 월요일인지에 따라 첫째 주를 정의해야 합니다. 영어권에서...,https://kin.naver.com/qna/detail.naver?d1id=6&...
21152,산모와 태아가 혈액형이 다를 때 문제가 없는 이유는 무엇인가요?,"산모와 태아가 탯줄과 태반을 통해 직접적으로 혈액을 교환하지 않고, 산모의 태반에서...",https://kin.naver.com/qna/detail.naver?d1id=7&...
21153,의자의 쇠 부분을 만지면 왜 정전기가 발생할까요?,의자의 쇠 부분을 만지면 정전기가 발생하는 이유는 의자가 땅에서 절연되어 있기 때문...,https://kin.naver.com/qna/detail.naver?d1id=11...


In [4]:
data['embedding'] = data.apply(lambda row: llm.encode(row.instruction), axis = 1)


AttributeError: 'GPT4All' object has no attribute 'encode'

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
# 코사인 유사도 구하기
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

def return_similar_answer(input):
    # 입력 텍스트를 임베딩 벡터로 변환
    embedding = model.encode(input)
    
    # `train_data` 데이터프레임의 각 행에 대해 유사도 점수를 계산
    # lambda 함수는 각 행의 'embedding' 열과 입력 임베딩 벡터 간의 코사인 유사도를 계산
    train_data['score']=train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    
    # 가장 높은 유사도 점수를 가진 행의 인덱스를 찾고, 해당 행의 'A' 열의 값을 반환
    # idxmax()는 score 열에서 최대값의 인덱스를 반환합니다
    # loc[]는 해당 인덱스의 행을 선택합니다
    return train_data.loc[train_data['score'].idxmax()]['A']


In [9]:
!conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch -c nvidia

^C


In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer
data = pd.read_parquet("C:/Users/humming/Downloads/GPT2_Chatbot/train-00000-of-00001-21df739eb88d711e.parquet")
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
data['embedding'] = data.apply(lambda row: model.encode(row.instruction), axis = 1)

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))
def return_answer(question):
    embedding = model.encode(question)
    data['score'] = data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return data.loc[data['score'].idxmax()]['output']

: 